## Scraping presidential visits by Russian presidents

The visits are announced in English on this site http://en.kremlin.ru/events/president/trips. I used BeautifulSoup for scraping the pages.

In [1]:
#importing libraries
import pandas as pd
%matplotlib inline
import requests
from bs4 import BeautifulSoup
import time

In [26]:
#finding the content of each enty on Kremlin webpage from all the pages after the year 2000.
def get_trip_links(page_number):
    print(page_number)
    r = requests.get("http://en.kremlin.ru/events/president/trips/page/"+str(page_number))
    soup = BeautifulSoup(r.content)
    links = soup.find_all("a")
    content_div = soup.find("div", {"class": "entry-content"})
    content_links = content_div.find_all("a")
    trip_links = []
    for cl in content_links:
        if "trips" in cl["href"]:
            if cl["href"] in trip_links:
                pass
            else:
                trip_links.append(cl["href"])
    return trip_links

In [27]:
def get_trip_dicts(trip_links):
    trip_dicts = []
    for url_end in trip_links:
        url2 = "http://en.kremlin.ru" + url_end
        doc=requests.get(url2).content
        #Including a timer to stop the requests every two seconds to keep the page we are scraping happy.
        time.sleep(2)
        soup_doc = BeautifulSoup(doc,"html.parser")
    
        dict_page = {}
        #there are some None values in the location that we want as empty rows.
        if soup_doc.find(rel="tag") is None:
            dict_page['location'] = ""
        else:
            #I want to se everything as pure text as possible
            dict_page['location'] = soup_doc.find(rel="tag").text
        dict_page['date']= soup_doc.find_all(itemprop="datePublished")[1].string
        dict_page['title']=soup_doc.find_all("h1")[1].text
        dict_page['link']= url2
        trip_dicts.append(dict_page)
    return trip_dicts

In [29]:
#This goes through all the page numbers, it would be 1-43 but now we use the range defined above.     
trip_lists = []
i = 1
while i <= 43:
    trip_links = get_trip_links(i)
    time.sleep(5)
    trip_lists.append(get_trip_dicts(trip_links))
    i += 1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


In [31]:
all_trips = [item for tl in trip_lists for item in tl]

In [32]:
all_trips

[{'location': 'Vologda Region',
  'date': 'February 4, 2020, 13:10 ',
  'title': 'Vladimir Putin arrives in\xa0Cherepovets ',
  'link': 'http://en.kremlin.ru/events/president/trips/62730'},
 {'location': 'State of Palestine',
  'date': 'January 23, 2020, 21:00 ',
  'title': 'Meeting with President of\xa0Palestine Mahmoud Abbas ',
  'link': 'http://en.kremlin.ru/events/president/trips/62662'},
 {'location': 'Israel',
  'date': 'January 23, 2020, 10:15 ',
  'title': 'Vladimir Putin arrives in\xa0Israel ',
  'link': 'http://en.kremlin.ru/events/president/trips/62661'},
 {'location': 'Lipetsk Region',
  'date': 'January 22, 2020, 13:20 ',
  'title': 'Vladimir Putin toured social infrastructure facilities in\xa0Usman District, Lipetsk Region ',
  'link': 'http://en.kremlin.ru/events/president/trips/62659'},
 {'location': 'Germany',
  'date': 'January 19, 2020, 14:40 ',
  'title': 'Vladimir Putin arrived in\xa0Berlin to\xa0attend international conference on\xa0Libya ',
  'link': 'http://en.k

## Save the Data

In [33]:
#creating a datarame of the dictionary
df = pd.DataFrame(all_trips)

In [34]:
df.head()

,location,date,title,link
0,Vologda Region,"February 4, 2020, 13:10",Vladimir Putin arrives in Cherepovets,http://en.kremlin.ru/events/president/trips/62730
1,State of Palestine,"January 23, 2020, 21:00",Meeting with President of Palestine Mahmoud Ab...,http://en.kremlin.ru/events/president/trips/62662
2,Israel,"January 23, 2020, 10:15",Vladimir Putin arrives in Israel,http://en.kremlin.ru/events/president/trips/62661
3,Lipetsk Region,"January 22, 2020, 13:20",Vladimir Putin toured social infrastructure fa...,http://en.kremlin.ru/events/president/trips/62659
4,Germany,"January 19, 2020, 14:40",Vladimir Putin arrived in Berlin to attend int...,http://en.kremlin.ru/events/president/trips/62616


In [35]:
#saving the frame as csv file
df.to_csv('russia-trips.csv', index=False)

As a result, we have a csv file with location country of the visit, the date of the visit, the reason of the visit and the link that gives more information of the visit. I can add the name of the president just by checking the time of the visit (there have only been two between 2000-2019). 

In the final outcome there are also visits inside the Russia so they need to be filtered out. I used Google sheets for that.

---

---

---